In [60]:
import csv

filename = "assignment_3_dataset.csv"

with open(filename, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Read the header row
    print("Headers:", header)
    
    # Print first 5 rows
    for i, row in enumerate(reader):
        print(row)
        if i == 4:  # Stop after 5 rows
            break

Headers: ['', 'person', 'lang', 'z1000t', 'z0t', 'z1000mem', 'stmtL', 'z1000rel', 'm1000rel', 'whours', 'caps']
['0', 's018', 'c', '0.017', '0.017', '22432', '16.1', '98.1', '96.8', '16.1', '??']
['1', 's030', 'C', '', '0.033', '16968', '4.0', '76.47', '92.1', '4.0', '??']
['2', 's036', 'c', '20.9', '0.0', '11440', '8.2', '0.0', '89.5', '8.2', '??']
['3', 's066', 'C', '0.75', '0.467', '2952', '7.3', '98.48', '100.0', '7.3', '??']
['4', 's078', 'c', '0.05', '0.05', '22496', '10.9', '99.24', '98.4', '10.9', '??']


In [61]:
import pandas as pd

df = pd.read_csv('assignment_3_dataset.csv')

# Display all rows nicely
from IPython.display import display
display(
    df.head(10).style.set_table_styles([
        {'selector': 'th', 'props': [('border', '1px solid black')]},
        {'selector': 'td', 'props': [('border', '1px solid gray')]}
    ])
)


,Unnamed: 0,person,lang,z1000t,z0t,z1000mem,stmtL,z1000rel,m1000rel,whours,caps
0,0,s018,c,0.017000,0.017000,22432,16.100000,98.100000,96.800000,16.100000,??
1,1,s030,C,nan,0.033000,16968,4.000000,76.470000,92.100000,4.000000,??
2,2,s036,c,20.900000,0.000000,11440,8.200000,0.000000,89.500000,8.200000,??
3,3,s066,C,0.750000,0.467000,2952,7.300000,98.480000,100.000000,7.300000,??
4,4,s078,c,0.050000,0.050000,22496,10.900000,99.240000,98.400000,10.900000,??
5,5,s015,c++,0.050000,0.050000,24616,11.200000,99.240000,100.000000,11.200000,??
6,6,s020,C++,1.983000,0.550000,6384,3.000000,98.480000,98.400000,3.000000,??
7,7,s021,c++,4.867000,0.017000,5312,nan,100.000000,98.400000,nan,??
8,8,s025,c++,0.083000,0.083000,28568,3.500000,99.240000,98.400000,3.500000,??
9,9,s027,c++,1.533000,0.000000,3472,25.300000,98.090000,100.000000,25.300000,??


## Functions
- Declare functions here - can be called in later cells
- ensure function cell has been run

In [62]:
# Function to color-code reliability values
def highlight_reliability(val):
    """Color-code reliability values based on performance ranges."""
    if pd.isna(val):
        return ''
    elif val < 10:
        return 'color: red; font-weight: bold;'
    elif val < 25:
        return 'color: orange;'
    elif val < 50:
        return 'color: goldenrod;'
    elif val < 90:
        return 'color: lightgreen;'
    elif val <= 100:
        return 'color: green; font-weight: bold;'
    else:
        return ''

## Missing Values
- Columns with missing values: ...
- Decide how to handle them: drop rows, fill with mean/median, etc.

In [63]:
df.isna().sum()

Unnamed: 0    0
person        0
lang          0
z1000t        5
z0t           0
z1000mem      0
stmtL         3
z1000rel      0
m1000rel      0
whours        3
caps          2
dtype: int64

## Step 2: Data Cleaning

Before analyzing the dataset, it is important to ensure the data is complete and consistent. This step involves:

1. **Handling Missing Values**  
   - We check each column for missing values (`NaN`).  
   - Any rows with missing values are removed using `dropna()`.  
   - This ensures that calculations and visualizations are not affected by incomplete data.

2. **Creating a Cleaned Dataset**  
   - The cleaned dataset is stored in `df_clean`.  
   - We confirm that all missing values are removed by checking the sum of NaNs per column.


In [64]:
import numpy as np

# Drop 'Unnamed: 0' column if it exists

df_dropped = df.copy()

if 'Unnamed: 0' in df.columns:
    df_dropped = df.drop(columns=['Unnamed: 0'])

if 'caps' in df_dropped.columns:
    df_dropped = df_dropped.drop(columns=['caps'])

# Normalize the language names
df_dropped['lang'] = df_dropped['lang'].str.strip().str.lower()

# Drop rows with any NaN values
df_clean = df_dropped.dropna()

# Sort by 'lang' ascending and 'whours' descending
df_sorted = df_clean.sort_values(by=['lang', 'whours'], ascending=[True, False])

# Reorder columns with caps at the end
#df_sorted = df_sorted[['person', 'lang', 'z1000t', 'z0t', 'z1000mem', 'stmtL', 'whours', 'z1000rel', 'm1000rel', 'caps']]

# Compute the 98th percentile (cap value)
loc_cap = df_sorted['whours'].quantile(0.98)

# Create a mask for rows below or equal to the cap
loc_mask = df_sorted['whours'] <= loc_cap

# Initialize Hours_std as NaN
df_sorted['Hours_std'] = np.nan

# Clip Hours values at the cap (optional, for display or further analysis)
df_sorted['Hours_cap'] = df_sorted['whours'].clip(upper=loc_cap)

# Standardize only the non-outlier rows
df_sorted.loc[loc_mask, 'Hours_std'] = (df_sorted.loc[loc_mask, 'Hours_cap'] - df_sorted.loc[loc_mask, 'Hours_cap'].mean()) / df_sorted.loc[loc_mask, 'Hours_cap'].std()

# Reorder columns without 'caps'
df_sorted = df_sorted[['person', 'lang', 'z1000t', 'z0t', 'z1000mem', 'stmtL', 'whours','Hours_std', 'z1000rel', 'm1000rel']]

# Rename columns for clarity
df_renamed = df_sorted.rename(columns={
    'person': 'ID',
    'lang': 'language',
    'z1000t': 'run_z1000_time',
    'z0t': 'run_z0_time',
    'stmtL': 'LOC',
    'z1000mem': 'mem_z1000_kb',
    'whours': 'hours',
    'caps': 'self_rated'
})

# Limit displayed decimal places to 2
pd.options.display.float_format = "{:.2f}".format

# Identify numeric columns
numeric_cols = df_renamed.select_dtypes(include='number').columns

# Print row counts before and after cleaning
print("Original rows:", len(df))
print("Rows after dropping NaNs:", len(df_renamed))


# Display cleaned DataFrame
from IPython.display import display
display(
    df_renamed.head(10)
    .style.format({col: "{:.2f}" for col in numeric_cols})
    # Highlight 'z1000rel' and 'm1000rel' columns with color-coding based on reliability
    .map(highlight_reliability, subset=['z1000rel', 'm1000rel'])
    .set_table_styles([
        {'selector': 'th', 'props': [('border', '1px solid black')]},
        {'selector': 'td', 'props': [('border', '1px solid gray')]}
    ])
)


Original rows: 80
Rows after dropping NaNs: 73


,ID,language,run_z1000_time,run_z0_time,mem_z1000_kb,LOC,hours,Hours_std,z1000rel,m1000rel
0,s018,c,0.02,0.02,22432.00,16.10,16.10,0.67,98.10,96.80
4,s078,c,0.05,0.05,22496.00,10.90,10.90,0.19,99.24,98.40
2,s036,c,20.90,0.00,11440.00,8.20,8.20,-0.07,0.00,89.50
3,s066,c,0.75,0.47,2952.00,7.30,7.30,-0.15,98.48,100.00
9,s027,c++,1.53,0.00,3472.00,25.30,25.30,1.53,98.09,100.00
14,s090,c++,1.67,0.03,4152.00,19.60,19.60,1.00,98.48,100.00
13,s051,c++,0.15,0.03,3448.00,15.20,15.20,0.59,100.00,98.40
12,s042,c++,0.03,0.03,22680.00,11.90,11.90,0.28,100.00,100.00
5,s015,c++,0.05,0.05,24616.00,11.20,11.20,0.21,99.24,100.00
10,s033,c++,0.03,0.03,23336.00,10.10,10.10,0.11,99.24,98.40


# Exploratory Data Analysis
- Calculate and present descriptive statistics 
- Visualize distributions of key variables.
- Describe any anomalies or potential biases in the data.

In [ ]:
numeric_cols = df_renamed.select_dtypes(include='number').drop(columns=['Hours_std']).columns

# Descriptive statistics
for lang in df_renamed['language'].unique():
    print(f"Descriptive statistics for language: {lang}")
    lang_df = df_renamed[df_renamed['language'] == lang]
    descriptive_count = lang_df[numeric_cols].agg(['count']).iloc[0, 0]  # Extract only the first value
    print("Count:", descriptive_count)  # Print the count directly
    descriptive_stats = lang_df[numeric_cols].agg(['mean', 'min', 'max'])
    display(descriptive_stats)

Descriptive statistics for language: c
Count: 4


,run_z1000_time,run_z0_time,mem_z1000_kb,LOC,hours,Hours_std,z1000rel,m1000rel
mean,5.43,0.13,14830.00,10.62,10.62,0.16,73.95,96.17
min,0.02,0.00,2952.00,7.30,7.30,-0.15,0.00,89.50
max,20.90,0.47,22496.00,16.10,16.10,0.67,99.24,100.00


Descriptive statistics for language: c++
Count: 10


,run_z1000_time,run_z0_time,mem_z1000_kb,LOC,hours,Hours_std,z1000rel,m1000rel
mean,2.78,0.09,12876.00,11.42,11.42,0.23,89.28,89.47
min,0.03,0.00,3448.00,3.00,3.00,-0.55,0.00,1.10
max,21.40,0.55,28568.00,25.30,25.30,1.53,100.00,100.00


Descriptive statistics for language: java
Count: 24


...

,run_z1000_time,run_z0_time,mem_z1000_kb,LOC,hours,Hours_std,z1000rel,m1000rel
mean,6.43,0.35,16153.33,3.21,3.21,-0.53,99.68,76.43
min,0.08,0.07,13632.00,1.50,1.50,-0.69,99.24,0.00
max,72.30,1.37,22608.00,5.50,5.50,-0.32,100.00,100.00


Descriptive statistics for language: rexx
Count: 3


,run_z1000_time,run_z0_time,mem_z1000_kb,LOC,hours,Hours_std,z1000rel,m1000rel
mean,15.74,0.72,17088.00,6.31,6.31,-0.24,81.46,81.30
min,0.82,0.30,8968.00,0.93,0.93,-0.74,44.75,46.60
max,25.40,0.95,21152.00,10.00,10.00,0.10,100.00,98.90


Descriptive statistics for language: tcl
Count: 8


,run_z1000_time,run_z0_time,mem_z1000_kb,LOC,hours,Hours_std,z1000rel,m1000rel
mean,29.53,0.64,29662.00,4.72,4.72,-0.39,100.00,88.78
min,0.47,0.37,17672.00,0.60,0.60,-0.77,100.00,10.20
max,202.80,1.43,48840.00,13.00,13.00,0.38,100.00,100.00


In [65]:
import sys
print(sys.executable)


C:\Users\isakk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
